<a href="https://colab.research.google.com/github/marianoogimenez/google-asl-project/blob/master/recommendation-system/collaborative-filtering/create-wals-dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Collaborative filtering using WALS algorithm  | 2

## Create walls dataset

In [0]:
PROJECT = 'qwiklabs-gcp-4a684069c4776675'
BUCKET = 'colaborative-filtering-agea'
REGION = 'us-central1'

In [0]:
from google.colab import auth
import pandas as pd
import numpy as np

In [0]:
auth.authenticate_user()

### Mapping

In [0]:
def create_mapping(values, filename):
  with open(filename, 'w') as ofp:
    value_to_id = {value:idx for idx, value in enumerate(values.unique())}
    for value, idx in value_to_id.items():
      ofp.write('{},{}\n'.format(value, idx))
  return value_to_id


In [6]:
!gsutil cp gs://{BUCKET}/collab_raw.csv  'data/collab_raw.csv'  

Copying gs://colaborative-filtering-agea/collab_raw.csv...
/ [1 files][ 21.5 KiB/ 21.5 KiB]                                                
Operation completed over 1 objects/21.5 KiB.                                     


In [0]:
df = pd.read_csv('data/collab_raw.csv',
                 header=None,
                 names=['visitorId', 'contentId', 'rating'],
                dtype={'visitorId': str, 'contentId': str, 'rating': np.float})

In [0]:
user_mapping = create_mapping(df['visitorId'], 'data/users.csv')
item_mapping = create_mapping(df['contentId'], 'data/items.csv')

In [9]:
#copy to bucket
!gsutil cp 'data/users.csv'  gs://{BUCKET}/users.csv   
!gsutil cp 'data/items.csv'  gs://{BUCKET}/items.csv  

Copying file://data/users.csv [Content-Type=text/csv]...
/ [1 files][ 11.6 KiB/ 11.6 KiB]                                                
Operation completed over 1 objects/11.6 KiB.                                     
Copying file://data/items.csv [Content-Type=text/csv]...
/ [1 files][ 12.0 KiB/ 12.0 KiB]                                                
Operation completed over 1 objects/12.0 KiB.                                     


In [0]:
df['userId'] = df['visitorId'].map(user_mapping.get)
df['itemId'] = df['contentId'].map(item_mapping.get)

In [11]:
mapped_df = df[['userId', 'itemId', 'rating']]
mapped_df.to_csv('data/collab_mapped.csv', index=False, header=False)
mapped_df.head()

,userId,itemId,rating
0,0,0,0.3
1,1,1,0.3
2,2,2,0.3
3,3,3,0.3
4,4,4,0.3
